In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import sqlite3

def_path = '/content/drive/Shareddrives/데이터톤/보고서 작성 시 사용할 최종 파일/데이터 및 전처리/'

#	라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	혼잡도  건물
raw_data_lounge = pd.read_csv(def_path+'라운지+혼잡도/라운지 데이터 최종.csv',encoding='euc-kr')
raw_data_lounge['건물'] = raw_data_lounge['건물'].replace({' ':''}, regex=True)

# 랜드마크 간의 거리 구하기
distance_buildings = pd.read_csv(def_path+'건물별 이동시간 최종.csv',encoding='utf-8')

encoding_df = pd.read_csv(def_path+'건물 인코딩.csv')

def df_completion(user):
    user_building = user[0]
    distance = distance_buildings[['building', user_building]].copy()
    distance.rename(columns = {user_building: '거리'}, inplace = True)

    #라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	혼잡도	건물	***거리
    lounge_with_distance = pd.merge(raw_data_lounge, distance, left_on= '건물', right_on='building', how ='left' ).drop(columns=['building'])

    day = str(pd.to_datetime(user[1].split('T')[0]).dayofweek)
    congestion_path = def_path+'라운지+혼잡도/혼잡도_'
    user_c = pd.read_csv(congestion_path+'월.csv')

    #find day
    if day==1:
        user_c =  pd.read_csv(congestion_path+'화.csv')
    elif day==2:
        user_c = pd.read_csv(congestion_path+'수.csv')
    elif day==3:
        user_c = pd.read_csv(congestion_path+'목.csv')
    elif day==4:
        user_c = pd.read_csv(congestion_path+'금.csv')

    range = pd.date_range("09:00", "23:30", freq="30min").strftime('%H:%M:%S')
    dt = pd.to_datetime(user[1]).round('30min')
    user_c_r = range.get_loc(str(dt).split(' ')[1].split('+')[0])

    #라운지	위치(층)	책상 높이	테이블 수	의자 수	냉난방장치	콘센트 수	창문	컴퓨터	정수기	화장실	***혼잡도	건물    거리
    lounge_with_congestion = lounge_with_distance
    lounge_with_congestion['혼잡도']=lounge_with_congestion.apply(lambda row: user_c[row.건물][user_c_r], axis=1)

    # classroom encoding (건물이름 --> 숫자)
    final_lounge = pd.merge(lounge_with_congestion, encoding_df, left_on= '건물', right_on='building', how ='left')
    final_lounge = final_lounge.drop(['building','위치(층)'], axis =1)
    final_lounge = final_lounge.rename({'건물':'building_name','라운지':'lounge_name','encoding':'건물', 'lounge_index':'라운지'},axis=1)
    print(final_lounge)
    return final_lounge

# 모델 --> 없으면 모델을 불러올 수가 없음
class NNCollabFiltering(nn.Module):
    def __init__(self, emb_size=10, n_hidden=100):
        super(NNCollabFiltering, self).__init__()
        self.emb_size = emb_size
        self.user_emb = nn.Embedding(100, emb_size)
        self.item_emb = nn.Embedding(600, emb_size)
        self.lin1 = nn.Linear(1000, n_hidden)
        self.lin2 = nn.Linear(n_hidden, 1)
        self.drop1 = nn.Dropout(0.1)
        
    def forward(self, u, v):
        U = self.user_emb(u)
        U = torch.reshape(U, (239, 6 * self.emb_size))
        V = self.item_emb(v)
        V = torch.reshape(V, (239, 12 * self.emb_size))
        x = F.relu(torch.cat([U, V], dim=1))
        x = self.drop1(x)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        x = torch.flatten(x)
        return x

def fetch_model(theme):
    # 모델 불러오기
    PATH = def_path+'라운지+혼잡도/model_'

    if theme in range (1,4):
        PATH = PATH + str(theme) + '/'
    else:
        PATH = PATH + '0/'

    model = torch.load(PATH + 'model.pt')  
    model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay= 1e-6)

    checkpoint = torch.load(PATH + 'all.tar')  
    model.load_state_dict(checkpoint['model'])
    optimizer.load_state_dict(checkpoint['optimizer'])

    return model


# softmax 함수 (약간의 추천 랜덤성 부여를 위해 사용)
def softmax(a) :
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y


###############       9개 라운지 추천해주는 모델      ########################
def lounge_Top9(theme, location, time, num_user, table, plug, computer, facility):
  
  # 인원 수, 시간 정보 사용 안함 
  new_user = [location, time, num_user, table, plug, computer, facility]

  lounge_df_norm = df_completion(new_user)
  lounge_df = lounge_df_norm.drop(['building_name','lounge_name'], axis =1)

  building_row = encoding_df.loc[encoding_df['building'] == location]
  encoding = building_row.iloc[0]['encoding']
  new_user[0] = encoding
  new_user.pop(1)

  
  model = fetch_model(theme)
  # 학습된 모델을 이용해 각 라운지에 대한 사용자 점수 계산
  with torch.no_grad():
    user_scale = np.array(new_user*239).reshape((239,6))
    user_input = torch.LongTensor(user_scale)
    l = lounge_df.to_numpy()
    lounge_input = torch.LongTensor(l[:,1:])
    out = model(user_input, lounge_input).numpy()
    sort_out = np.argsort(out)[::-1]
    sort_out_score = out[sort_out]
  
  # Softmax 추천 알고리즘
  top9_index = np.random.choice(sort_out, 9, p=softmax(sort_out_score), replace=False)
  top9_df = pd.DataFrame(top9_index, columns =['라운지'])

  top9 = pd.merge(top9_df, lounge_df_norm, on= '라운지', how ='left')
  output = top9[['building_name', 'lounge_name']].copy().to_records(index=False)
    
  return output 
####################################################################################


result = lounge_Top9(1, '국제관', '2021-08-09T15:00:00+09:00',5,5,2,1,0)
print(result)

FileNotFoundError: ignored